# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset
pokemon = pd.read_csv('Pokemon.csv')

In [4]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [6]:
# your code here
# H0: Total = HP + Attack + Defense + Sp. Att + Sp. Def + Speed
#What we want to do is calculate the sum of all the stats of all the pokemons and create a new column with Total_Calc.
#We then want to compare this with the already existing Total Column.

pokemon['Total_calc'] = pokemon['HP'] + pokemon['Attack'] + pokemon['Defense'] + pokemon['Sp. Atk'] + pokemon['Sp. Def'] + pokemon['Speed']



In [11]:
total_calc_sum = pokemon.Total_calc.sum()
total_sum = pokemon.Total.sum()

if total_calc_sum == total_sum:
    print(f"The totals are equal, namely {total_calc_sum} and {total_sum}")
else:
    print("Find another way, totals are not equal")

The totals are equal, namely 348082 and 348082


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)


In [17]:
# your code here

print(pokemon['Type 1'].unique())
print(pokemon['Type 1'].value_counts())


['Grass' 'Fire' 'Water' 'Bug' 'Normal' 'Poison' 'Electric' 'Ground'
 'Fairy' 'Fighting' 'Psychic' 'Rock' 'Ghost' 'Ice' 'Dragon' 'Dark' 'Steel'
 'Flying']
Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64


In [18]:
print(pokemon['Type 2'].unique())
print(pokemon['Type 2'].value_counts())

['Poison' nan 'Flying' 'Dragon' 'Ground' 'Fairy' 'Grass' 'Fighting'
 'Psychic' 'Steel' 'Ice' 'Rock' 'Dark' 'Water' 'Electric' 'Fire' 'Ghost'
 'Bug' 'Normal']
Flying      97
Ground      35
Poison      34
Psychic     33
Fighting    26
Grass       25
Fairy       23
Steel       22
Dark        20
Dragon      18
Water       14
Ghost       14
Ice         14
Rock        14
Fire        12
Electric     6
Normal       4
Bug          3
Name: Type 2, dtype: int64


In [25]:
one_hot_pokemons = pd.get_dummies(pokemon, columns = ['Type 1', 'Type 2'])
print(one_hot_pokemons.head())

   #                   Name  Total  HP  Attack  Defense  Sp. Atk  Sp. Def  \
0  1              Bulbasaur    318  45      49       49       65       65   
1  2                Ivysaur    405  60      62       63       80       80   
2  3               Venusaur    525  80      82       83      100      100   
3  3  VenusaurMega Venusaur    625  80     100      123      122      120   
4  4             Charmander    309  39      52       43       60       50   

   Speed  Generation  ...  Type 2_Ghost  Type 2_Grass  Type 2_Ground  \
0     45           1  ...             0             0              0   
1     60           1  ...             0             0              0   
2     80           1  ...             0             0              0   
3     80           1  ...             0             0              0   
4     65           1  ...             0             0              0   

   Type 2_Ice  Type 2_Normal  Type 2_Poison  Type 2_Psychic  Type 2_Rock  \
0           0              0

## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [30]:

#I feel this is wrong lol but i guess its Dragon pokemon, which is the coolest pokemon anyways so i'll pick the dragon type!
corr_pokemons = one_hot_pokemons.corrwith(one_hot_pokemons['Total'])
ranked_corr_pokemons = corr_pokemons.sort_values(ascending=False)
print(ranked_corr_pokemons)

Total              1.000000
Total_calc         1.000000
Sp. Atk            0.747250
Attack             0.736211
Sp. Def            0.717609
HP                 0.618748
Defense            0.612787
Speed              0.575943
Legendary          0.501758
Type 1_Dragon      0.196532
Type 2_Fighting    0.138726
#                  0.119813
Type 2_Dragon      0.115240
Type 2_Ice         0.100870
Type 1_Psychic     0.094364
Type 1_Steel       0.082000
Type 2_Psychic     0.076054
Type 2_Fire        0.073234
Type 2_Steel       0.070307
Type 2_Dark        0.065844
Type 2_Flying      0.054048
Type 1_Fire        0.050527
Generation         0.048384
Type 1_Rock        0.037524
Type 1_Flying      0.029504
Type 1_Dark        0.017818
Type 1_Electric    0.016715
Type 2_Ground      0.016486
Type 2_Electric    0.014669
Type 1_Ghost       0.007594
Type 1_Ground      0.004082
Type 2_Rock       -0.000512
Type 1_Ice        -0.002412
Type 2_Ghost      -0.004885
Type 2_Normal     -0.013956
Type 1_Water      -0

C:\Users\geert\AppData\Local\Temp\ipykernel_18040\183945257.py:2: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_pokemons = one_hot_pokemons.corrwith(one_hot_pokemons['Total'])


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# probably calculate the total correlations of all the type 1 and compare with all the type 2 but i think its statistically incorrect
